# Data Salaries
* How much money do software engineers make?
* What about Data Scientists?
* Which companies are paying the most?
* What are the trends looking like?

In [90]:
import pandas as pd
import requests
import numpy as np
import math
from datetime import timedelta

#data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [51]:
data = requests.get('https://www.levels.fyi/js/salaryData.json').json()
df = pd.DataFrame(data)

## Clean Data

In [52]:
# Remove columns that we don't need
df = df.drop(['cityid','rowNumber','dmaid'], axis=1)
df = df.replace("", np.nan)

#convert datatypes
num_cols = ['yearsofexperience','basesalary','bonus','stockgrantvalue',
            'totalyearlycompensation','yearsatcompany']
df[num_cols] = df[num_cols].apply(pd.to_numeric)

#one record without a location, kick it out
df = df[df.location.notnull()]

#round up all of the years of experience even if it is 0.25 years
df['yearsofexperience'] = np.ceil(df.yearsofexperience)
df['yearsatcompany'] = np.ceil(df.yearsatcompany)

#remove records that fall in the top/bottom 95th/5th percentile on totalyearly compensation
#I do this to remove some of the submissions that say they are making $5 million a year or those that are next to nothing
df = df[df['totalyearlycompensation'].between(df['totalyearlycompensation']. \
                                              quantile(.05),df['totalyearlycompensation'].quantile(.95))]

#remove records that are outside of the US. This definition is any location record that has 2 commas or more
df = df[df['location'].str.count(',') == 1]

In [53]:
df.shape

(16572, 14)

#### Clean up the date submission data

In [54]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['SubmitDate'] = pd.to_datetime(df['timestamp'])

In [55]:
# Get Date Function
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [56]:
submit_year, submit_month, _ = get_date_int(df, 'SubmitDate')

In [68]:
#Normalize dates to the beginning of the month
df['SubmitMonth'] = pd.to_datetime(submit_year *10000 + submit_month * 100 + 1, format='%Y%m%d')
df['SubmitWeek'] = pd.to_datetime(df['SubmitDate'].dt.date) - ((df['SubmitDate'].dt.weekday) + 1).astype('timedelta64[D]')

## Observe the flow of submissions over time

In [69]:
df_time = df.SubmitWeek.value_counts().reset_index()
df_time.columns = ['Date','Count']
df_time = df_time.sort_values(by='Date')

In [79]:
df_time['RollingCount'] = df_time['Count'].rolling(window=6).mean()

In [93]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_time['Date'],
                         y=df_time['RollingCount'],
                         mode='lines',
                         name='Submission Count'))
fig.update_layout(title='Count of Submissions by Week (6 week moving average)')

#add 10 days to the end of the x axis
fig['layout']['xaxis'].update(range=[min(df_time['Date']),max(df_time['Date']) + timedelta(days=10)])

fig.show()

## What are the top most recent submissions

In [117]:
recent = df.groupby(by=['SubmitMonth'], as_index=False).agg({'title':'nunique','yearsofexperience':'mean',
                                                           'totalyearlycompensation':['mean','count']})
recent.columns = ['SubmitMonth','Unique Titles','AvgExp','AvgComp','Count']
recent.tail(10)

SubmitMonth  Unique Titles    AvgExp     AvgComp  Count
23  2019-05-01             91  7.622905  238.317039    716
24  2019-06-01            140  7.814599  225.805839   1370
25  2019-07-01             86  7.058568  226.247289    922
26  2019-08-01             92  7.118943  223.724670    908
27  2019-09-01            110  7.367765  229.382903   1123
28  2019-10-01             80  6.031056  218.056936    966
29  2019-11-01             89  5.739631  214.080645    868
30  2019-12-01             95  6.907828  228.606061    792
31  2020-01-01            105  7.161800  229.972376   1267
32  2020-02-01            143  7.537143  225.789388   1225

In [104]:
recent.Title.value_counts()

Software Engineer                33
Software Engineering Manager     33
Product Manager                  33
Solution Architect               21
Data Scientist                   21
                                 ..
Senior Software engineer          1
GPS                               1
Cognitive Software Developer      1
Director of QA                    1
Services Solutions Specialist     1
Name: Title, Length: 920, dtype: int64

In [119]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=recent['SubmitMonth'],
                         y=recent['Unique Titles'],
                         mode='lines',
                         name='Submission Count'))
fig.update_layout(title='Count of Submissions by Week (6 week moving average)')

#add 10 days to the end of the x axis
#fig['layout']['xaxis'].update(range=[min(df_time['Date']),max(df_time['Date']) + timedelta(days=10)])

fig.show()

## Look at the top 50 submitted Companies

In [19]:
#get the top 50 companies
top_50_list = df.company.value_counts(ascending=False).head(50).index.tolist()
df_50 = df[df.company.isin(top_50_list)]

In [20]:
#top 50 submissions - grouped by company - and less than 10 years of experience

df_50_10 = df_50.loc[df_50.yearsofexperience < 10]
df_50_10 = df_50_10.groupby(by='company', as_index=False)['totalyearlycompensation'].\
                                                        agg({'totalyearlycompensation':['count','mean','median','max']})
#rename multi-index columns
df_50_5.columns = ['Company','Count','Mean','Median','Max']
#round the data to 1 decimal place
df_50_5.iloc[:,-3:] = df_50_5.iloc[:,-3:].round(1)

In [17]:
#df_50[df_50.yearsofexperience <= 5]['yearsofexperience'].value_counts()

In [18]:
#df_50.location.value_counts().tail(10)

## How has pay by tenure changed over time?

In [22]:
df_experience = df[df.yearsofexperience <= 5].groupby(by=['SubmitDate','yearsofexperience'],
                                                        as_index=False)['totalyearlycompensation'].\
                                                        agg({'totalyearlycompensation':['count','mean','median','max']})
df_experience.columns = ['SubmitDate','YearsExp','Count','Mean','Median','Max']
#start in June 2018 because of sample sizes
df_experience = df_experience[df_experience.SubmitDate > '2018-05-01']
df_experience.sample(5)

SubmitDate  YearsExp  Count        Mean  Median    Max
89  2018-09-01       5.0     58  226.620690   217.0  450.0
118 2019-02-01       4.0     38  209.763158   203.5  355.0
165 2019-10-01       3.0     92  194.554348   178.0  410.0
189 2020-02-01       3.0    109  203.155963   187.0  495.0
85  2018-09-01       1.0     44  171.000000   163.5  315.0

In [23]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_experience[df_experience['YearsExp'] == 0]['SubmitDate'],
                         y=df_experience[df_experience['YearsExp'] == 0]['Mean'],
                         mode='lines+markers',
                         name='0 Years'))

fig.add_trace(go.Scatter(x=df_experience[df_experience['YearsExp'] == 1]['SubmitDate'],
                         y=df_experience[df_experience['YearsExp'] == 1]['Mean'],
                         mode='lines+markers',
                         name='1 Year'))

fig.add_trace(go.Scatter(x=df_experience[df_experience['YearsExp'] == 2]['SubmitDate'],
                         y=df_experience[df_experience['YearsExp'] == 2]['Mean'],
                         mode='lines+markers',
                         name='2 Years'))

fig.add_trace(go.Scatter(x=df_experience[df_experience['YearsExp'] == 3]['SubmitDate'],
                         y=df_experience[df_experience['YearsExp'] == 3]['Mean'],
                         mode='lines+markers',
                         name='3 Years'))

fig.add_trace(go.Scatter(x=df_experience[df_experience['YearsExp'] == 4]['SubmitDate'],
                         y=df_experience[df_experience['YearsExp'] == 4]['Mean'],
                         mode='lines+markers',
                         name='4 Years'))

fig.add_trace(go.Scatter(x=df_experience[df_experience['YearsExp'] == 5]['SubmitDate'],
                         y=df_experience[df_experience['YearsExp'] == 5]['Mean'],
                         mode='lines+markers',
                         name='5 Years'))




fig.update_layout(title='Average Annual Compenstion by Years of Experience')

fig.show()

## What about Data Science?

In [120]:
df_ds = df[df.title == 'Data Scientist']

In [121]:
def q25(x):
    return x.quantile(0.25)
def q75(x):
    return x.quantile(0.75)

In [122]:
df_ds_exp = df_ds.groupby(by=['yearsofexperience'],
                          as_index=False).agg({'totalyearlycompensation':['count','mean','max','min','median',q25, q75]})
df_ds_exp.columns = ['Years of Experience','Count','Mean','Max','Min','Median','bottom25%','top25%']
df_ds_exp.iloc[:,2:] = round(df_ds_exp.iloc[:,2:],1)
df_ds_exp.head(6)

Years of Experience  Count   Mean    Max    Min  Median  bottom25%  top25%
0                  0.0     55  187.4  368.0  100.0   189.0      150.0   214.0
1                  1.0     55  186.9  332.0   99.0   168.0      139.5   225.0
2                  2.0     73  185.3  420.0   95.0   177.0      141.0   223.0
3                  3.0     69  197.6  423.0  100.0   200.0      147.0   232.0
4                  4.0     58  225.7  500.0  104.0   210.0      180.2   270.0
5                  5.0     64  215.7  360.0  118.0   207.0      174.5   250.0

In [123]:
top_titles = df.title.value_counts().head(7).index.tolist()
top_titles

['Software Engineer',
 'Product Manager',
 'Software Engineering Manager',
 'Data Scientist',
 'Hardware Engineer',
 'Product Designer',
 'Solution Architect']

In [124]:
df_titles = df[df.title.isin(top_titles)]
df_titles = df_titles[df_titles.yearsofexperience <= 5]
df_titles.loc[df_titles.title == 'Data Scientist', 'DataScientist'] = 1
df_titles['DataScientist'] = df_titles['DataScientist'].fillna(0)
df_titles['DataScientist'] = df_titles['DataScientist'].astype(int)
df_titles.DataScientist.value_counts()

0    7241
1     374
Name: DataScientist, dtype: int64

In [125]:
df_titles.title.value_counts()

Software Engineer               6474
Data Scientist                   374
Product Manager                  365
Hardware Engineer                192
Product Designer                 138
Software Engineering Manager      47
Solution Architect                25
Name: title, dtype: int64

In [126]:
fig = go.Figure()
 
fig.add_trace(go.Box(
    x= df_titles[df_titles.DataScientist == 1 ]['title'],
    y= df_titles[df_titles.DataScientist == 1 ]['totalyearlycompensation'],
    name='Data Scientist',
    marker_color= "#002B5C"))

fig.add_trace(go.Box(
    x= df_titles[df_titles.DataScientist == 0 ]['title'],
    y= df_titles[df_titles.DataScientist == 0 ]['totalyearlycompensation'],
    name='Other',
    marker_color= "#00471B"))

fig.update_layout(
    title="Compensation for Top 10 Titles (<6 yrs exp)",
    yaxis_title="Total Yearly Compensation (thousands)",
    font=dict(
        family="Roboto",
        size=14,
        color="#696969"
    )
)


fig.show()

In [31]:
df.sort_values(by='timestamp', ascending=False)

timestamp    company             level  \
18517 2020-02-12 18:56:04   Qualcomm  Senior Staff Eng   
18515 2020-02-12 17:46:43  Bloomberg           Manager   
18514 2020-02-12 17:24:03     Amazon            SDE II   
18513 2020-02-12 17:03:49      Cisco          Grade 11   
18512 2020-02-12 16:44:32     Amazon                L5   
...                   ...        ...               ...   
6     2017-06-17 00:23:14      Apple                M1   
4     2017-06-14 21:22:25  Microsoft                64   
2     2017-06-11 14:53:57     Amazon                L7   
1     2017-06-10 17:11:29       eBay              SE 2   
0     2017-06-07 11:33:27     Oracle                L3   

                              title  totalyearlycompensation  \
18517         Senior Staff Manager                     285.0   
18515  Software Engineering Manager                    375.0   
18514             Software Engineer                    200.0   
18513             Software Engineer                    180.0   
18512             Software Engineer                    185.0   
...                             ...                      ...   
6      Software Engineering Manager                    372.0   
4      Software Engineering Manager                    200.0   
2                   Product Manager                    310.0   
1                 Software Engineer                    100.0   
0                   Product Manager                    127.0   

                location  yearsofexperience  yearsatcompany  \
18517      San Diego, CA               12.0            12.0   
18515       New York, NY               18.0            13.0   
18514       New York, NY               15.0             9.0   
18513       San Jose, CA               10.0             2.0   
18512        Herndon, VA                7.0             3.0   
...                  ...                ...             ...   
6          Sunnyvale, CA                7.0             5.0   
4            Redmond, WA                9.0             9.0   
2            Seattle, WA                8.0             0.0   
1      San Francisco, CA                5.0             3.0   
0       Redwood City, CA                2.0             2.0   

                                  tag  basesalary  stockgrantvalue    bonus  \
18517                           Modem       168.0             85.0     32.0   
18515  Distributed Systems (Back-End)       300.0              5.0     70.0   
18514  Distributed Systems (Back-End)       130.0             70.0      0.0   
18513                      Full Stack       150.0             10.0     20.0   
18512          Site Reliability (SRE)       145.0             40.0      0.0   
...                               ...         ...              ...      ...   
6                                 NaN       157.0            180.0     35.0   
4                                 NaN    169000.0         100000.0  30000.0   
2                                 NaN       155.0              NaN      NaN   
1                                 NaN         NaN              NaN      NaN   
0                                 NaN       107.0             20.0     10.0   

      gender otherdetails SubmitDate  SubmitYear  SubmitMonth  
18517   Male      Masters 2020-02-01        2020            2  
18515   Male      Masters 2020-02-01        2020            2  
18514   Male      Masters 2020-02-01        2020            2  
18513    NaN      Masters 2020-02-01        2020            2  
18512  Other          NaN 2020-02-01        2020            2  
...      ...          ...        ...         ...          ...  
6        NaN          NaN 2017-06-01        2017            6  
4        NaN          NaN 2017-06-01        2017            6  
2        NaN          NaN 2017-06-01        2017            6  
1        NaN          NaN 2017-06-01        2017            6  
0        NaN          NaN 2017-06-01        2017            6  

[15829 rows x 17 columns]

# Men Women Gap

In [127]:
df_gender = df[(df['gender'] == 'Male') | (df['gender'] == 'Female')]
df_gender = df_gender[df_gender.title.isin(top_titles)]

In [128]:
df_genderpay = df_gender.groupby(by=['gender','title'], 
                  as_index=False).agg({'totalyearlycompensation':'mean',
                                       'yearsofexperience':['mean','count']})
df_genderpay.columns = ['gender','title','compmean','yearsexp','count']
df_genderpay_pivot = df_genderpay.pivot(index='title', columns='gender', values='compmean').reset_index()
df_genderpay_pivot['Diff'] = df_genderpay_pivot['Female'] - df_genderpay_pivot['Male']
df_genderpay_pivot = df_genderpay_pivot.sort_values(by='Diff')
df_genderpay_pivot

gender                         title      Female        Male       Diff
3                    Product Manager  219.148571  254.582878 -35.434307
6                 Solution Architect  205.666667  239.682635 -34.015968
1                  Hardware Engineer  192.000000  223.148607 -31.148607
2                   Product Designer  200.534091  228.212500 -27.678409
5       Software Engineering Manager  295.700000  310.318618 -14.618618
4                  Software Engineer  209.240561  221.731588 -12.491027
0                     Data Scientist  216.487179  221.706061  -5.218881

In [129]:
#df_genderpay[df_genderpay['gender'] == 'Male']['compmean']

In [130]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_genderpay_pivot['Male'],
    y=df_genderpay_pivot['title'],
    marker=dict(color="crimson", size=12),
    mode="markers",
    name="Male",
))

fig.add_trace(go.Scatter(
    x=df_genderpay_pivot['Female'],
    y=df_genderpay_pivot['title'],
    marker=dict(color="blue", size=12),
    mode="markers",
    name="Female",
))

fig.update_layout(title="Gender Earnings Disparity",
                  xaxis_title="Total Compensation",
                  yaxis_title="Job Title")

fig.show()

In [164]:
df.loc[df.location.str[-2:] == 'UT']

timestamp        company                                level  \
153   2017-11-20 14:52:29           Snap                                   L3   
894   2018-06-21 10:22:44       Ancestry                                Staff   
3822  2018-12-05 20:16:21    Pluralsight                            Mid level   
4589  2019-01-17 07:07:25          Adobe                                    4   
4986  2019-02-04 12:04:07          Adobe                  Software Engineer 3   
5118  2019-02-09 13:49:04       Facebook                                  IC4   
5422  2019-02-23 12:17:22           SoFi                               Senior   
6750  2019-04-10 15:52:27     Salesforce                                    7   
8291  2019-06-05 08:16:06    Pluralsight                                   P4   
8371  2019-06-07 12:11:13          Adobe            Site Reliability Engineer   
8848  2019-06-15 22:08:30    Pluralsight                                    3   
8908  2019-06-16 07:10:14    Wells Fargo                  Software Engineer 5   
9281  2019-06-19 07:07:48           SoFi                           Senior SWE   
9760  2019-06-29 20:29:57       Ancestry           Sr Infrastructure Engineer   
10121 2019-07-12 08:49:41    Pluralsight                            Mid-level   
10244 2019-07-15 22:10:58          Adobe                                    4   
10814 2019-07-30 12:10:09          Adobe         Senior Software Engineer (4)   
10857 2019-07-31 14:10:58       Ancestry                      Senior Engineer   
10917 2019-08-01 12:04:12          Adobe                  Software Engineer 3   
11287 2019-08-15 17:06:28      Qualtrics                                   L5   
11318 2019-08-16 09:01:35       Symantec                                 SE 5   
11397 2019-08-17 21:38:34    Pluralsight                                   L2   
11721 2019-08-27 08:51:11       Symantec             Senior Software Engineer   
11794 2019-08-28 20:31:00          Adobe                                   L4   
11812 2019-08-29 10:33:07       raytheon                                   L5   
12488 2019-09-14 08:59:56         Amazon                                   L5   
12702 2019-09-16 21:30:06  Goldman Sachs                                 NAPA   
13046 2019-09-25 20:54:54         Amazon                                   L6   
13652 2019-10-09 23:47:18          Adobe                    Product manager 2   
13697 2019-10-11 08:06:06       Ancestry                               Senior   
14153 2019-10-24 17:23:53       Symantec                                 SE 3   
14850 2019-11-13 15:59:52      Qualtrics                                   L3   
16729 2020-01-09 15:24:08  Goldman Sachs                            Associate   
16788 2020-01-11 04:35:38    Pluralsight                                   P3   
16947 2020-01-15 09:17:49      Qualtrics                                   L5   
17390 2020-01-22 18:37:58    Pluralsight                                   P3   
17507 2020-01-24 12:02:39      Qualtrics                                   L4   
17728 2020-01-28 16:31:49      Qualtrics                                   L3   
18910 2020-02-15 17:53:28      Qualtrics                                   L4   
19228 2020-02-20 13:09:29    Pluralsight                                   P4   
19306 2020-02-21 18:50:52  Goldman Sachs  Executive Director / Vice-President   

                              title  totalyearlycompensation  \
153               Software Engineer                    188.0   
894               Software Engineer                    106.0   
3822               Product Designer                    105.0   
4589               Network Engineer                    151.0   
4986              Software Engineer                    156.0   
5118               Network Engineer                    130.0   
5422              Software Engineer                    141.0   
6750              Software Engineer                    139.0   
8291               Product Designer  

In [154]:
df.location.dtype

dtype('O')